In [1]:
import numpy as np
import pandas as pd
import jieba
import json
import jieba.analyse
from tqdm import tqdm

In [2]:
data = pd.read_csv('./近30日文章内容.csv')

### 1 提取Title+内容

In [5]:
#单个样本测试
#json.loads(data.c_content[0])[0].get('content')

'下一个视频应该就是飙车剪辑了'

In [3]:
#提取文章内容
def extractContent(data):
    if str(data)=='nan':
        text = json.loads(json.dumps([{"content":None}]))
    else:
        text = json.loads(data)
    content=''
    for i in text:
        sentence = i.get('content')
        if sentence:
            content += sentence
    return content

In [4]:
tqdm.pandas(desc="my bar!")
data['content'] = data.c_content.progress_apply(lambda x: extractContent(x))

C:\Users\JDD\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
my bar!: 100%|████████████████████████████████████████████████████████████████| 65105/65105 [00:01<00:00, 35566.26it/s]


In [5]:
data.drop(columns=['c_content'],inplace=True)

In [6]:
#title * 2
data['content'] = data['c_title']*2+data['content']

In [10]:
#保存数据
#data.to_csv('./articles_recent30days.csv',encoding='utf-8_sig')

### 2 数据预处理

In [1]:
import numpy as np
import pandas as pd
import jieba
import json
import jieba.analyse
from collections import Counter
import math

In [2]:
#data = pd.read_csv('./key_words.csv')

In [9]:
data

,itemid,c_title,content
0,2130495,小忍者日本佛系跑山第二弹！夜跑神户市章山！【Moto vlog】,小忍者日本佛系跑山第二弹！夜跑神户市章山！【Moto vlog】小忍者日本佛系跑山第二弹！夜...
1,3609545,阳光明媚的一天,阳光明媚的一天阳光明媚的一天
2,1636262,卖400万的摩托会飞？没错！它就是一台飞行摩托车，空陆两用,卖400万的摩托会飞？没错！它就是一台飞行摩托车，空陆两用卖400万的摩托会飞？没错！它就是...
3,3594525,警钟长鸣 又一江西少年 过弯死亡！,警钟长鸣 又一江西少年 过弯死亡！警钟长鸣 又一江西少年 过弯死亡！关键词 ...
4,869674,铃木SFV650改装箭牌排气，声浪听着怎么样？有没有想驾驭它的冲动,铃木SFV650改装箭牌排气，声浪听着怎么样？有没有想驾驭它的冲动铃木SFV650改装箭牌排...
...,...,...,...
65100,2783852,没事洗洗...,没事洗洗...没事洗洗...
65101,2665627,焕新本色，多样生活！金城新款提拉米苏解析,焕新本色，多样生活！金城新款提拉米苏解析焕新本色，多样生活！金城新款提拉米苏解析金城的提拉米...
65102,3694511,【4K】川崎机械增压街车Z H2 山路骑行第一视角,【4K】川崎机械增压街车Z H2 山路骑行第一视角【4K】川崎机械增压街车Z H2 山路骑行...
65103,2824784,慢慢骑~骑到老~骑在邯郸,慢慢骑~骑到老~骑在邯郸慢慢骑~骑到老~骑在邯郸感谢金球国际影院提供的免费场次！邯郸C1D大...


In [3]:
#data = data.loc[data.content.notnull(),:]
#data = data.reset_index(drop=True)

In [11]:
#设置停用词
jieba.analyse.set_stop_words('./stopwords.txt')

In [12]:
#设置userdict
jieba.load_userdict('./user_dict.txt')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\JDD\AppData\Local\Temp\jieba.cache
Loading model cost 0.677 seconds.
Prefix dict has been built succesfully.


In [13]:
#设置idf path
jieba.analyse.set_idf_path('./idf.txt')

In [14]:
#设置stop_words
stop_words = set((
        "the", "of", "is", "and", "to", "in", "that", "we", "for", "an", "are",
        "by", "be", "as", "on", "with", "can", "if", "from", "which", "you", "it",
        "this", "then", "at", "have", "all", "not", "one", "has", "or", "that"
    ))
stop_content = open('./stopwords.txt', 'rb').read().decode('utf-8')
for line in stop_content.splitlines():
    stop_words.add(line)

In [15]:
#提取最终关键词
def extract_tokens(data):
    tfidf_keywords = tf_idf_(data)
    textrank_keywords = textrank_(data)
    tfidf_dict = {}
    textrank_dict = {}
    for i in range(len(tfidf_keywords)):
        tfidf_dict[tfidf_keywords[i][0]] = tfidf_keywords[i][1]
    for i in range(len(textrank_keywords)):
        textrank_dict[textrank_keywords[i][0]] = textrank_keywords[i][1]
    key_words=tfidf_dict.keys()&textrank_dict.keys()
    keywords_dict ={}
    for word in key_words:
        keywords_dict[word] = tfidf_dict[word]*textrank_dict[word]
    tokens = sorted(keywords_dict.keys(),key=lambda x: x[1], reverse=True)
    return tokens

#提取带权重的关键词
def extract_keywords_weight(data):
    tfidf_keywords = tf_idf_(data)
    textrank_keywords = textrank_(data)
    tfidf_dict = {}
    textrank_dict = {}
    for i in range(len(tfidf_keywords)):
        tfidf_dict[tfidf_keywords[i][0]] = tfidf_keywords[i][1]
    for i in range(len(textrank_keywords)):
        textrank_dict[textrank_keywords[i][0]] = textrank_keywords[i][1]
    key_words=tfidf_dict.keys()&textrank_dict.keys()
    keywords_dict ={}
    for word in key_words:
        keywords_dict[word] = tfidf_dict[word]*textrank_dict[word]
    keywords_weight = sorted(keywords_dict.items(),key=lambda x: x[1], reverse=True)
    
    return keywords_weight

#提取tf-idf关键词
def tf_idf_(data):
    return jieba.analyse.extract_tags(data,topK=20,withWeight=True)

#提取textrank关键词
def textrank_(data):
    return jieba.analyse.textrank(data,topK=20,withWeight=True)

In [ ]:
%%time
data['tfidf权重'] = data.content.progress_apply(lambda x: tf_idf_(x))

my bar!:  94%|██████████████████████████████████████████████████████████████▎   | 61411/65105 [02:45<00:10, 341.56it/s]

In [28]:
%%time
data['textrank权重'] = data.content.progress_apply(lambda x: textrank_(x))

Wall time: 26min 32s


In [29]:
%%time
data['tokens'] = data.content.progress_apply(lambda x: extract_tokens(x))

Wall time: 25min 56s


In [30]:
%%time
data['最终关键词权重']=data.content.progress_apply(lambda x: extract_keywords_weight(x))

Wall time: 26min 13s


In [31]:
data.head()

,Unnamed: 0,itemid,c_title,content,tokens,tfidf权重,textrank权重,最终关键词权重
0,0,2130495,小忍者日本佛系跑山第二弹！夜跑神户市章山！【Moto vlog】,小忍者日本佛系跑山第二弹！夜跑神户市章山！【Moto vlog】小忍者日本佛系跑山第二弹！夜...,"[视频, 剪辑, 飙车, 佛系, 日本]","[(夜跑, 0.8678290392654254), (vlog, 0.6962554812...","[(飙车, 1.0), (日本, 0.9997540318932343), (视频, 0.9...","[(佛系, 0.6911610009081428), (剪辑, 0.350146702081..."
1,1,3609545,阳光明媚的一天,阳光明媚的一天阳光明媚的一天,[],"[(阳光明媚, 6.205054621719346)]",[],[]
2,2,1636262,卖400万的摩托会飞？没错！它就是一台飞行摩托车，空陆两用,卖400万的摩托会飞？没错！它就是一台飞行摩托车，空陆两用卖400万的摩托会飞？没错！它就是...,"[驾驶员, 体重, 道路, 飞行, 品牌, 喷气, 人民币, 引擎, 模式, 车型]","[(飞行, 0.4588183382184316), (喷气, 0.120959974280...","[(飞行, 1.0), (车型, 0.5646018610501963), (品牌, 0.4...","[(飞行, 0.4588183382184316), (车型, 0.037810096363..."
3,3,3594525,警钟长鸣 又一江西少年 过弯死亡！,警钟长鸣 又一江西少年 过弯死亡！警钟长鸣 又一江西少年 过弯死亡！关键词 ...,"[驾驶, 关键词, 限速, 超速, 后车, 机车, 江西, 指示牌, 游玩, 路段, 过弯,...","[(超速, 0.2339563168612277), (死亡, 0.222465450483...","[(驾驶, 1.0), (机车, 0.8409865271792483), (事发, 0.5...","[(驾驶, 0.12605706336773156), (事发, 0.10480506648..."
4,4,869674,铃木SFV650改装箭牌排气，声浪听着怎么样？有没有想驾驭它的冲动,铃木SFV650改装箭牌排气，声浪听着怎么样？有没有想驾驭它的冲动铃木SFV650改装箭牌排...,"[驾驭, 视频, 改装, 转自, 声浪, 有没有, 排气, 头条, 冲动]","[(SFV650, 0.7138918694533889), (冲动, 0.53835865...","[(冲动, 1.0), (改装, 0.8417816757587867), (驾驭, 0.7...","[(冲动, 0.5383586548967971), (驾驭, 0.279933085379..."


In [32]:
data.to_csv('./key_words.csv',encoding='utf-8_sig')